In [80]:
import numpy as np
from matplotlib import pyplot as plt
import PIL
import cv2
import time
import pickle
import methods as mt
import sys
from munkres import Munkres, print_matrix

In [2]:
trees = []
roots = []

nodesList = [[]]
edgesList = [[]]

roots = mt.importMolecule('gxl')


listedMolecules = mt.moleculeToList(roots)
nodesList = listedMolecules[0]
edgesList = listedMolecules[1]
print(nodesList[0])
print(edgesList[0])

[10071, ['O', 'C', 'C', 'C', 'O', 'C', 'C', 'C', 'C', 'C', 'O', 'C']]
[10071, [[1, 2], [1, 3], [3, 4], [3, 5], [4, 6], [4, 7], [5, 8], [6, 9], [7, 10], [8, 11], [9, 12], [6, 8], [10, 12]]]


In [3]:
nodesDict = mt.moleculeNodeListToDict(nodesList)
edgesDict = mt.moleculeEdgeListToDict(edgesList)
print(edgesDict[10071])
# print(nodesDict)
# sort dictionnary by node id (if this is necessary)
for i in sorted(nodesDict) :  
     print ((i, nodesDict[i]), end =" ")

[[1, 2], [1, 3], [3, 4], [3, 5], [4, 6], [4, 7], [5, 8], [6, 9], [7, 10], [8, 11], [9, 12], [6, 8], [10, 12]]
(16, ['P', 'O', 'N', 'O', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']) (35, ['O', 'C', 'C', 'N', 'C', 'N', 'C', 'N', 'N', 'N', 'N']) (40, ['N', 'C', 'C', 'C', 'C', 'C', 'Cl', 'Cl']) (91, ['C', 'S', 'C', 'O', 'O', 'O', 'O']) (94, ['C', 'S', 'S', 'C', 'C', 'S']) (124, ['C', 'C', 'S', 'C', 'O', 'O', 'O', 'C', 'C', 'C', 'C', 'C']) (125, ['C', 'C', 'S', 'C', 'O', 'O', 'O', 'C', 'O', 'C', 'C']) (126, ['C', 'S', 'C', 'O', 'O', 'O', 'C', 'C', 'O', 'C', 'C', 'C']) (166, ['N', 'O', 'C', 'C', 'C', 'C', 'C', 'C', 'O']) (171, ['O', 'S', 'O', 'O', 'C', 'C']) (188, ['S', 'C', 'O', 'O', 'C', 'C', 'N', 'C', 'C', 'O', 'N', 'O']) (194, ['C', 'C', 'N', 'S', 'C', 'C', 'C', 'S', 'C']) (200, ['C', 'C', 'N', 'C', 'S', 'O', 'O', 'C', 'O', 'O', 'O']) (228, ['S', 'C', 'C', 'N', 'N', 'C', 'C', 'N', 'N', 'N']) (244, ['S', 'O', 'O', 'O', 'C', 'O', 'C', 'C', 'C', 'C', 'O', 'O']) (251, ['N', 'C', 'N', 'S', 'N', '

## KNN Algorithm:
1. compare every molecule (from the valid set) with every other molecule (from the train set)
 - calculate each time the GED (graph edit distance between the two molecules)
 - to be more precise: approximate GED using bipartite graph matching
 - build cost matrix (using Dirac cost function (see lecture 9 slide 36))
 - use Hungarian algorithm to find optimal assignment (using an existing framework!)
2. for ever molecule (from the valid set) find the K closest molecules (in the train set)
 - the K molecules with the shortest GED (since we only have two classes (active, inactive): use odd K!)
 - using train.txt check if the KNN are active or inactive
 - assign molecule to the closer class (and check if assignment was correct using valid.txt (for accuracy measure))
 - also: optimize for K (task for later)

Current TODO: create cost matrix for two molecules

In [25]:
Cn = 1
Ce = 1

In [75]:
# lecture 10 slide 36
def BP(molecule1, molecule2, edges1, edges2):
    
    #1. build dirac cost matrix
    matrix = np.matrix(np.ones((len(molecule1)+len(molecule2),len(molecule1)+len(molecule2))) * np.inf)
    #Upper left: substitutions
    for i in range(0, len(molecule1)):
        for j in range(0, len(molecule2)):
            if not(molecule1[i] == molecule2[j]):
                val = Ce*abs(edges1[i] - edges2[j]) #number of node deletions/insertions (times the cost)
                matrix.itemset((i,j),2*Cn + val)            
            else:
                val = Ce*abs(edges1[i] - edges2[j]) #number of node deletions/insertions
                matrix.itemset((i,j), val)
    #Upper right: deletions
    for i in range(0, len(molecule1)):
        val = Cn + Ce*edges1[i] #cost of deleting the node plus number of nodes times cost of deleting all edges of this node
        matrix.itemset((i,len(molecule2) +i), val)
    #Lower left: insertions
    for i in range(0, len(molecule2)):
        val = Cn + Ce*edges2[i] #cost of inserting the node plus number of nodes times cost of inserting all edges of this node
        matrix.itemset((len(molecule1)+i,i), val)
    #Lower right: zeros
    for i in range(0, len(molecule2)):
        for j in range(0, len(molecule1)):
            matrix.itemset((len(molecule1) + i,len(molecule2) + j),0) 
            
    return matrix
    
    #2. find optimal assignment (using Hungarian algorithm)
    
    #3. calculate edit path distance/cost (of the optimal assignment)
    
    #4. return distance/cost

In [89]:
print(nodesList[0])
print(nodesList[1])
nr = len(nodesDict[edgesList[0][0]])
nr2 = len(nodesDict[edgesList[1][0]])
print(moleculeEdgesToCount(nr, edgesList[0][1]))
print(moleculeEdgesToCount(nr2, edgesList[1][1]))
print(sys.maxsize)

#get cost matrix (of two molecules)
m = BP(nodesList[0][1], nodesList[1][1], moleculeEdgesToCount(nr, edgesList[0][1]), moleculeEdgesToCount(nr2, edgesList[1][1]))
print(m)
#replace infinity with max int
mat = np.where(m == np.inf, sys.maxsize, m)
print(mat.tolist())
#Hungarian algorithm (get total cost)
matrix = mat.tolist()
m = Munkres()
indexes = m.compute(matrix)
print_matrix(matrix, msg='Lowest cost through this matrix:')
total = 0
for row, column in indexes:
    value = matrix[row][column]
    total += value
    print(f'({row}, {column}) -> {value}')
print(f'total cost: {total}')

[10071, ['O', 'C', 'C', 'C', 'O', 'C', 'C', 'C', 'C', 'C', 'O', 'C']]
[10085, ['N', 'O', 'C', 'C', 'C', 'N', 'N', 'O', 'N', 'C', 'C']]
[2, 1, 3, 3, 2, 3, 2, 3, 2, 2, 1, 2]
[2, 1, 3, 3, 3, 1, 2, 1, 2, 3, 1]
9223372036854775807
[[ 2.  1.  3.  3.  3.  3.  2.  1.  2.  3.  3.  3. inf inf inf inf inf inf
  inf inf inf inf inf]
 [ 3.  2.  2.  2.  2.  2.  3.  2.  3.  2.  0. inf  2. inf inf inf inf inf
  inf inf inf inf inf]
 [ 3.  4.  0.  0.  0.  4.  3.  4.  3.  0.  2. inf inf  4. inf inf inf inf
  inf inf inf inf inf]
 [ 3.  4.  0.  0.  0.  4.  3.  4.  3.  0.  2. inf inf inf  4. inf inf inf
  inf inf inf inf inf]
 [ 2.  1.  3.  3.  3.  3.  2.  1.  2.  3.  3. inf inf inf inf  3. inf inf
  inf inf inf inf inf]
 [ 3.  4.  0.  0.  0.  4.  3.  4.  3.  0.  2. inf inf inf inf inf  4. inf
  inf inf inf inf inf]
 [ 2.  3.  1.  1.  1.  3.  2.  3.  2.  1.  1. inf inf inf inf inf inf  3.
  inf inf inf inf inf]
 [ 3.  4.  0.  0.  0.  4.  3.  4.  3.  0.  2. inf inf inf inf inf inf inf
   4. inf inf inf inf

In [55]:
[[ 2.  1.  3.  3.  3.  3.  2.  1.  2.  3.  3.  3. inf inf inf inf inf inf inf inf inf inf inf]
 [ 3.  2.  2.  2.  2.  2.  3.  2.  3.  2.  0. inf  2. inf inf inf inf inf inf inf inf inf inf]
 [ 3.  4.  0.  0.  0.  4.  3.  4.  3.  0.  2. inf inf  4. inf inf inf inf inf inf inf inf inf]
 [ 3.  4.  0.  0.  0.  4.  3.  4.  3.  0.  2. inf inf inf  4. inf inf inf inf inf inf inf inf]
 [ 2.  1.  3.  3.  3.  3.  2.  1.  2.  3.  3. inf inf inf inf  3. inf inf inf inf inf inf inf]
 [ 3.  4.  0.  0.  0.  4.  3.  4.  3.  0.  2. inf inf inf inf inf  4. inf inf inf inf inf inf]
 [ 2.  3.  1.  1.  1.  3.  2.  3.  2.  1.  1. inf inf inf inf inf inf  3. inf inf inf inf inf]
 [ 3.  4.  0.  0.  0.  4.  3.  4.  3.  0.  2. inf inf inf inf inf inf inf 4. inf inf inf inf]
 [ 2.  3.  1.  1.  1.  3.  2.  3.  2.  1.  1. inf inf inf inf inf inf inf inf  3. inf inf inf]
 [ 2.  3.  1.  1.  1.  3.  2.  3.  2.  1.  1. inf inf inf inf inf inf inf inf inf  3. inf inf]
 [ 3.  0.  4.  4.  4.  2.  3.  0.  3.  4.  2. inf inf inf inf inf inf inf inf inf inf  2. inf]
 [ 2.  3.  1.  1.  1.  3.  2.  3.  2.  1.  1. inf inf inf inf inf inf inf inf inf inf inf  3.]
 [ 3. inf inf inf inf inf inf inf inf inf inf  0.  0.  0.  0.  0.  0.  0. 0.  0.  0.  0.  0.]
 [inf  2. inf inf inf inf inf inf inf inf inf  0.  0.  0.  0.  0.  0.  0. 0.  0.  0.  0.  0.]
 [inf inf  4. inf inf inf inf inf inf inf inf  0.  0.  0.  0.  0.  0.  0. 0.  0.  0.  0.  0.]
 [inf inf inf  4. inf inf inf inf inf inf inf  0.  0.  0.  0.  0.  0.  0. 0.  0.  0.  0.  0.]
 [inf inf inf inf  4. inf inf inf inf inf inf  0.  0.  0.  0.  0.  0.  0. 0.  0.  0.  0.  0.]
 [inf inf inf inf inf  2. inf inf inf inf inf  0.  0.  0.  0.  0.  0.  0. 0.  0.  0.  0.  0.]
 [inf inf inf inf inf inf  3. inf inf inf inf  0.  0.  0.  0.  0.  0.  0. 0.  0.  0.  0.  0.]
 [inf inf inf inf inf inf inf  2. inf inf inf  0.  0.  0.  0.  0.  0.  0. 0.  0.  0.  0.  0.]
 [inf inf inf inf inf inf inf inf  3. inf inf  0.  0.  0.  0.  0.  0.  0. 0.  0.  0.  0.  0.]
 [inf inf inf inf inf inf inf inf inf  4. inf  0.  0.  0.  0.  0.  0.  0. 0.  0.  0.  0.  0.]
 [inf inf inf inf inf inf inf inf inf inf  2.  0.  0.  0.  0.  0.  0.  0. 0.  0.  0.  0.  0.]]

SyntaxError: invalid syntax (<ipython-input-55-4c58bab4e542>, line 1)

In [14]:
def moleculeEdgesToCount(nr, edgesList):
    count = list()
    for i in range(nr):
        count.append(sum(x.count(i+1) for x in edgesList))
    return count

In [16]:
print(edgesList[0][1])
print(len(nodesDict[edgesList[0][0]]))
nr = len(nodesDict[edgesList[0][0]])
print(moleculeEdgesToCount(nr, edgesList[0][1]))

[[1, 2], [1, 3], [3, 4], [3, 5], [4, 6], [4, 7], [5, 8], [6, 9], [7, 10], [8, 11], [9, 12], [6, 8], [10, 12]]
12
[2, 1, 3, 3, 2, 3, 2, 3, 2, 2, 1, 2]


In [58]:
matrix = [[5, 9, 1],
          [10, 3, 2],
          [8, 7, 4]]
m = Munkres()
indexes = m.compute(matrix)
print_matrix(matrix, msg='Lowest cost through this matrix:')
total = 0
for row, column in indexes:
    value = matrix[row][column]
    total += value
    print(f'({row}, {column}) -> {value}')
print(f'total cost: {total}')

Lowest cost through this matrix:
[ 5,  9,  1]
[10,  3,  2]
[ 8,  7,  4]
(0, 2) -> 1
(1, 1) -> 3
(2, 0) -> 8
total cost: 12
